In [4]:
'''
:@Author: Remi
:@Date: 2023/11/15 15:42:40
:@LastEditors: Remi
:@LastEditTime: 2023/12/14 10:18:25
:Description: 
'''
import pandas as pd
import seaborn as sns
import file_tool as ft
import numpy as np
import preprocessing as pp
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import ListedColormap
'''
保存格式：
eg：csv中

1223  1231  123  2323
123    213  123  23123

代表两个sample对应四个峰的峰面积
'''
################cache/需要改的
conc = []

cmap=mpl.cm.RdYlBu_r#获取色条
newcolors=cmap(np.linspace(0,1,256))#分片操作
newcmap=ListedColormap(newcolors[100:])#只取100之后的颜色列表，前面的舍去

np.set_printoptions(precision = 6)

peak_list = [779.898,852.075]##############peak position
tol = 1#算峰面积的容差


##################path##########################
filename = r'D:\20231218\normalized.csv'#file position
savepath = r'D:\20221218'
##################load#########################

wl,spec=ft.Load_MultiSpec(filename)

meanspec = np.mean(spec,axis = 1)############while variable no_grad use mean spec , otherwise chose max spec 
maxspec = np.max(spec,axis = 1)

p,m,peak_list = pp.Peak_integrate(wl,maxspec,peak_list,tol)

##################calculate#####################
for i in range(len(spec[0])):
    for j in range(len(peak_list)):#pm wavelength

        slice_cache = spec[peak_list[j] - m[j]:peak_list[j] + p[j],i]
        slice_square = (p[j] + m[j] + 1) * (slice_cache[0] + slice_cache[-1])/2
        if j == 0:
            slice_ = slice_cache.sum(axis=0) - slice_square
        else:
            slice_ = np.vstack((slice_,(slice_cache.sum(axis=0) - slice_square)))

        if i == 0:
            conc.append(wl[peak_list[j]])

    if i == 0:
        slice_sample = slice_   
    else: 
        slice_sample = np.hstack((slice_sample,slice_))

###################save#####################################

#np.savetxt(savepath +'\\' + 'square.csv',np.hstack((np.array([conc]).T,slice_sample)).T,delimiter=',',fmt='%.04f')  

###################plot#####################################

for j in range(len(peak_list)):
    sns_spec = spec[:][peak_list[j] - m[j]:peak_list[j] + p[j],:]
    sns_wl = wl_cache = wl[peak_list[j] - m[j]:peak_list[j] + p[j]]

##################减阴影
    for i in range(len(sns_spec[0,:])):
        slope = (sns_spec[:,i][0] - sns_spec[:,i][-1])/(sns_wl[0] - sns_wl[-1])
        bias = sns_spec[:,i][0] - slope * sns_wl[0]
        sns_spec[:,i] = sns_spec[:,i] - (sns_wl * slope + bias)
